In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [3]:
# rename columns

In [4]:
column_names = {'date':'Date',
                'Appliances':'Appliances_use',
                'lights':'light_use',
                'T1':'Temperature_kitchen',
                'RH_1':'Humidity_kitchen',
                'T2':'Temperature_living_room',
                'RH_2':'Humidity_living_room',
                'T3':'Temperature_laundry_room',
                'RH_3':'Humidity_laundry_room',
                'T4':'Temperature_office_room',
                'RH_4':'Humidity_office_room',
                'T5':'Temperature_bathroom',
                'RH_5':'Humidity_bathroom',
                'T6':'Temperature_outside_building',
                'RH_6':'Humidity_outside_building',
                'T7':'Temperature_ironing_room',
                'RH_7':'Humidity_ironing_room',
                'T8':'Temperature_teenage_room',
                'RH_8':'Humidity_teenage_room',
                'T9':'Temperature_parents_room',
                'RH_9':'Humidity_parents_room',
                'T_out':'Temperature_outside_Chievres',
                'press_mm_hg':'pressure_Chievres',
                'RH_out':'Humidity_outside_Chievres',
                'windspeed':'wind_speed',
                'visibility':'visibility_Chievres',
                'Tdewpoint':'Tdew_Chievres',
                'rv1':'random_variable_1',
                'rv2':'random_variable_2'}
df = df.rename(columns=column_names)

In [5]:
remove_date_light = df.drop(columns = ['Date', 'light_use'])

In [6]:
# select sample

In [7]:
def_reg = df[['Temperature_living_room', 'Temperature_outside_building']]

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
measure = MinMaxScaler()

In [10]:
print(def_reg)

       Temperature_living_room  Temperature_outside_building
0                    19.200000                      7.026667
1                    19.200000                      6.833333
2                    19.200000                      6.560000
3                    19.200000                      6.433333
4                    19.200000                      6.366667
...                        ...                           ...
19730                25.890000                     24.796667
19731                25.754000                     24.196667
19732                25.628571                     23.626667
19733                25.414000                     22.433333
19734                25.264286                     21.026667

[19735 rows x 2 columns]


In [11]:
normalised_df = pd.DataFrame(measure.fit_transform(def_reg),columns=def_reg.columns)

In [14]:
feature_df = normalised_df.drop(columns = ['Temperature_outside_building'])

In [15]:
target = normalised_df['Temperature_outside_building']

In [16]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(feature_df, target, test_size=0.3, random_state=42)

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
model = LinearRegression()

In [21]:
model.fit(x_train, y_train)

LinearRegression()

In [22]:
predicted_value = model.predict(x_test)

In [23]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_value)
round(r2_score, 3)

0.643

In [24]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_value)
round(mae,3)

0.082

In [26]:
import numpy as np

In [27]:
rss = np.sum(np.square(y_test - predicted_value))
round(rss, 2)

66.12

In [28]:
from sklearn.metrics import mean_squared_error
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, predicted_value))
round(rmse, 2)

0.11

In [29]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_value)
round(r2_score, 2)

0.64

In [30]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

In [31]:
def get_weight_df(model, feat, col_name):
    # this funtion returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features',col_name]
    weights_df[col_name].round(2)
    return weights_df

In [33]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [35]:
ridge_weights_df = get_weight_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weight_df(lasso_reg, x_train, 'Lasso_Weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [36]:
print(final_weights)

                  Features  Linear_Model_weights  Ridge_Weight  Lasso_Weight
0  Temperature_living_room              0.891077      0.889811       0.85178
